In [ ]:
conda install pandas

In [ ]:
conda install matplotlib

In [ ]:
conda install numpy

In [ ]:
conda install -c anaconda openpyxl

In [23]:
import pandas as pd
from IPython.display import FileLink

# Load the Excel file
file_path = "R3 L1 PM cellular distribution_8 bit.xlsx"
excel_data = pd.ExcelFile(file_path)

# List the sheet names in the file
sheet_names = excel_data.sheet_names

# NT analysis functions
def compile_all_regions_results(sheet_names, core_regions, shell_regions, channels):
    all_results = []
    for sheet in sheet_names:
        for region_group, region_names in {"CORE 1": core_regions, "SHELL 1": shell_regions, "SHELL 3": shell_regions}.items():
            for subregion in [f"LEFT {region_group}", f"RIGHT {region_group}"]:
                result = separate_and_calculate_nt(
                    sheet_name=sheet,
                    region_name=region_group,
                    specific_region=subregion,
                    channels=channels,
                )
                all_results.append(result)

            # Combine totals
            left_result = all_results[-2]
            right_result = all_results[-1]
            if left_result["total_rois"] != 0 and right_result["total_rois"] != 0:
                total_result = {
                    "sheet_name": sheet,
                    "region_name": region_group,
                    "specific_region": f"TOTAL {region_group}",
                    "total_rois": left_result["total_rois"] + right_result["total_rois"],
                    "nt_total": left_result["nt_total"] + right_result["nt_total"],
                    "nt_percentage": (
                        (left_result["nt_total"] + right_result["nt_total"]) /
                        (left_result["total_rois"] + right_result["total_rois"]) * 100
                    ),
                }
            else:
                total_result = {
                    "sheet_name": sheet,
                    "region_name": region_group,
                    "specific_region": f"TOTAL {region_group}",
                    "total_rois": "Data not available",
                    "nt_total": "Data not available",
                    "nt_percentage": "Data not available",
                }
            all_results.append(total_result)
    return all_results

def separate_and_calculate_nt(sheet_name, region_name, specific_region, channels):
    df = pd.read_excel(file_path, sheet_name=sheet_name)

    if 'Region' not in df.columns or 'ROI' not in df.columns:
        print(f"Missing columns in sheet {sheet_name}")
        return {
            "sheet_name": sheet_name,
            "region_name": region_name,
            "specific_region": specific_region,
            "total_rois": 0,
            "nt_total": 0,
            "nt_percentage": 0,
        }

    region_data = df[df['Region'] == specific_region]
    if region_data.empty:
        print(f"No data for region {specific_region} in sheet {sheet_name}")
        return {
            "sheet_name": sheet_name,
            "region_name": region_name,
            "specific_region": specific_region,
            "total_rois": 0,
            "nt_total": 0,
            "nt_percentage": 0,
        }

    grouped = region_data.groupby('ROI', group_keys=False)
    nt_total = grouped.apply(
        lambda group: set(channels).issubset(group['Channel'].unique()) and all(group['Max'] == 0),
        include_groups=False,
    ).sum()

    total_rois = region_data['ROI'].nunique()
    nt_percentage = (nt_total / total_rois * 100) if total_rois > 0 else 0

    return {
        "sheet_name": sheet_name,
        "region_name": region_name,
        "specific_region": specific_region,
        "total_rois": total_rois,
        "nt_total": nt_total,
        "nt_percentage": nt_percentage,
    }

# Perform NT analysis
combined_regions_results = compile_all_regions_results(
    sheet_names=sheet_names,
    core_regions=["LEFT CORE 1", "RIGHT CORE 1"],
    shell_regions=["LEFT SHELL 1", "RIGHT SHELL 1", "LEFT SHELL 3", "RIGHT SHELL 3"],
    channels=["D1DR", "D2DR", "OXTR"],
)

# Convert results to DataFrame
combined_regions_results_df = pd.DataFrame(combined_regions_results)

# Save the DataFrame to an Excel file
output_file_path = "NT_analysis_results.xlsx"
combined_regions_results_df.to_excel(output_file_path, index=False)
print(f"Results saved to {output_file_path}")

# Display a download link
FileLink(output_file_path)


No data for region LEFT SHELL 3 in sheet F128
No data for region RIGHT CORE 1 in sheet M160
No data for region RIGHT SHELL 1 in sheet M160
No data for region RIGHT SHELL 3 in sheet M160
Results saved to NT_analysis_results.xlsx


C:\Users\melo1934\NT_analysis_results.xlsx